In [1]:
import pandas as pd
import sys
import numpy as np
import pathlib as pl
import zipfile
sys.path.append('../dependencies/')
sys.path.append('../scripts/')
import pyemu
from datetime import datetime as dt
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import calendar
from postprocessing import plot_single_real
from post_process_model_output_fn import postprocess

/Users/mnfienen/Documents/GIT/hytest/NHM_CAL/nhm_pestpp/notebooks/../dependencies/pyemu/pst/pst_handler.py:15: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.


In [2]:
curr_model = '01473000'
# curr_model = '05431486'
# curr_model = '09112500'
curr_root = 'prior_mc_reweight'
citer = 0

In [3]:
tmp_res_path = pl.Path(f'../results/{curr_model}.prior_mc_reweight/')

In [4]:
postproc_nhm = False

if postproc_nhm:
    for cm in ['01473000','05431486','09112500']:
        print(f'postprocessing {cm}')
        c_path = pl.Path(f'../NHM_solutions/nhm_byHWobs_files/{curr_model}')
        postprocess(c_path, '.')

In [5]:
fig_dir = pl.Path(f'../postprocessing/figures/single_reals/{curr_model}')
if not fig_dir.exists():
    fig_dir.mkdir(parents=True)

In [6]:
res = pd.read_csv(tmp_res_path / f'{curr_root}.0.base.rei', delim_whitespace=True, skiprows=3, index_col=0)
res.loc[res.Measured<-999, 'Measured'] = np.nan
res.sample(5)

,Group,Measured,Modelled,Residual,Weight
Name,,,,,
l_max_sca_daily:2006_3_9:5628,l_max_sca_daily,NaN,0.000000,-9999.000000,0.00000
l_max_sca_daily:2005_4_18:5693,l_max_sca_daily,0.0,0.000000,0.000000,0.00000
streamflow_daily_4_1:2007_8_24:01472199,streamflow_daily_low,11.3,2.063093,9.236907,0.36954
g_min_sca_daily:2000_2_17:5693,g_min_sca_daily,NaN,0.872950,-9999.872950,0.00000
l_max_sca_daily:2002_6_3:5728,l_max_sca_daily,0.0,0.000000,0.000000,0.00000


In [7]:
res_nhm = pd.read_csv('../NHM_solutions/nhm_byHWobs_files/01473000/modelobs.dat', delim_whitespace=True, index_col=0)
res_nhm.sample(5)

,obsval
obsname,
streamflow_daily:2004_8_23:01472620,7.909827
g_min_sca_daily:2010_9_20:7157,0.000000
g_min_sca_daily:2004_3_27:5693,0.000000
l_max_runoff_mon:1983_11:7157,121.050685
g_min_runoff_mon:1988_12:5628,2.263249


In [8]:
newindex=[]
for i in res.index:
    if 'streamflow_daily' not in i:
        newindex.append(i)
    else:
        tmp = i.split(':')
        tmp[0]='streamflow_daily'
        newindex.append((':').join(tmp))
res.index=newindex

In [9]:
res.loc[res.Group==res.Group.unique()[-4]]

,Group,Measured,Modelled,Residual,Weight
streamflow_daily:2000_10_10:01472198,streamflow_daily_low,21.000000,7.444425,13.555575,0.198848
streamflow_daily:2000_10_10:01472199,streamflow_daily_low,13.000000,7.266691,5.733309,0.321216
streamflow_daily:2000_10_10:01472810,streamflow_daily_low,65.000000,45.731978,19.268022,0.064243
streamflow_daily:2000_10_10:01473000,streamflow_daily_low,151.000000,170.211870,-19.211870,0.027654
streamflow_daily:2000_10_11:01472198,streamflow_daily_low,21.000000,7.302361,13.697639,0.198848
...,...,...,...,...,...
streamflow_daily:2010_9_5:01472810,streamflow_daily_low,39.700001,3.774369,35.925632,0.105184
streamflow_daily:2010_9_6:01472810,streamflow_daily_low,40.400002,3.686997,36.713005,0.103362
streamflow_daily:2010_9_7:01472810,streamflow_daily_low,41.000000,3.601297,37.398703,0.101849
streamflow_daily:2010_9_8:01472810,streamflow_daily_low,40.500000,3.517254,36.982746,0.103106


In [10]:
res = res.merge(res_nhm.rename(columns={'obsval':'NHM_modelled'}), left_index=True, right_index=True)

In [11]:
res

,Group,Measured,Modelled,Residual,Weight,NHM_modelled
g_min_actet_mean_mon:10:5621,g_min_actet_mean_mon,0.026491,0.072515,-0.046024,2506.538170,0.058886
g_min_actet_mean_mon:10:5625,g_min_actet_mean_mon,0.035218,0.054129,-0.018910,2506.538170,0.052759
g_min_actet_mean_mon:10:5628,g_min_actet_mean_mon,0.027255,0.074992,-0.047738,2506.538170,0.065137
g_min_actet_mean_mon:10:5635,g_min_actet_mean_mon,0.036991,0.066819,-0.029828,2506.538170,0.066218
g_min_actet_mean_mon:10:5637,g_min_actet_mean_mon,0.034755,0.071988,-0.037233,2506.538170,0.063954
...,...,...,...,...,...,...
streamflow_mon:2010_9:01472198,streamflow_mon,15.563666,83.438703,-67.875037,6.719053,50.242661
streamflow_mon:2010_9:01472199,streamflow_mon,10.906666,52.928216,-42.021550,9.587999,37.426997
streamflow_mon:2010_9:01472620,streamflow_mon,36.813332,16.042930,20.770401,0.000000,19.902526
streamflow_mon:2010_9:01472810,streamflow_mon,51.806667,111.616052,-59.809385,2.018526,74.185287


In [12]:
res.Group.unique()

array(['g_min_actet_mean_mon', 'g_min_actet_mon', 'g_min_recharge_ann',
       'g_min_runoff_mon', 'g_min_sca_daily', 'g_min_soil_moist_ann',
       'g_min_soil_moist_mon', 'l_max_actet_mean_mon', 'l_max_actet_mon',
       'l_max_recharge_ann', 'l_max_runoff_mon', 'l_max_sca_daily',
       'l_max_soil_moist_ann', 'l_max_soil_moist_mon',
       'streamflow_daily_large_asc', 'streamflow_daily_small_asc',
       'streamflow_daily_small_dsc', 'streamflow_daily_pulse_asc',
       'streamflow_daily_pulse_dsc', 'streamflow_daily_low',
       'streamflow_daily_exlow', 'streamflow_mean_mon', 'streamflow_mon'],
      dtype=object)

In [13]:
cgroup = 'actet_mean_mon'
[plot_single_real(res, cgroup, citer, curr_root, fig_dir, curr_model) for cgroup in [
    # 'actet_mean_mon',
    # 'actet_mon',
    # 'recharge_ann',
    # 'soil_moist_mon',
    # 'soil_moist_ann',
    # 'runoff_mon',
    'streamflow_daily',
    'streamflow_mean_mon',
    'streamflow_mon'
]]

2000 01472198
2000 01472198
2000 01472198
2000 01472198
2000 01472198
2000 01472198
2000 01472198
2000 01472198
2000 01472198
2000 01472198
2000 01472198
2000 01472198
2001 01472198
2001 01472198
2001 01472198
2001 01472198
2001 01472198
2001 01472198
2001 01472198
2001 01472198
2001 01472198
2001 01472198
2001 01472198
2001 01472198
2002 01472198
2002 01472198
2002 01472198
2002 01472198
2002 01472198
2002 01472198
2002 01472198
2002 01472198
2002 01472198
2002 01472198
2002 01472198
2002 01472198
2003 01472198
2003 01472198
2003 01472198
2003 01472198
2003 01472198
2003 01472198
2003 01472198
2003 01472198
2003 01472198
2003 01472198
2003 01472198
2003 01472198
2004 01472198
2004 01472198
2004 01472198
2004 01472198
2004 01472198
2004 01472198
2004 01472198
2004 01472198
2004 01472198
2004 01472198
2004 01472198
2004 01472198
2005 01472198
2005 01472198
2005 01472198
2005 01472198
2005 01472198
2005 01472198
2005 01472198
2005 01472198
2005 01472198
2005 01472198
2005 01472198
2005 0

/Users/mnfienen/Documents/GIT/hytest/NHM_CAL/nhm_pestpp/notebooks/../scripts/postprocessing.py:423: MatplotlibDeprecationWarning: Keeping empty pdf files is deprecated since 3.8 and support will be removed two minor releases later.
/Users/mnfienen/Documents/GIT/hytest/NHM_CAL/nhm_pestpp/notebooks/../scripts/postprocessing.py:423: MatplotlibDeprecationWarning: Keeping empty pdf files is deprecated since 3.8 and support will be removed two minor releases later.


[None, None, None]